# Tarea 11 Computación cuántica: Implementación del algoritmo de búsqueda de Grovers.
Tarazona Jimenez Javier Andres
Muñoz Lemus Juan Sebastian


## Ejemplo Teleportación Cuántica (Es solo un ejemplo no se si sirva)

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Crear un circuito cuántico con 1 qubit y 1 bit clásico
circuit = QuantumCircuit(1, 1)

# Aplicar puerta Hadamard para crear superposición
circuit.h(0)

# Medir el qubit y guardar el resultado en el bit clásico
circuit.measure(0, 0)

# Mostrar el circuito
print("Circuito cuántico:")
print(circuit.draw())

# Ejecutar el circuito en el simulador Aer
simulador = Aer.get_backend('qasm_simulator')
resultado = execute(circuit, simulador, shots=1000).result()

# Obtener los conteos de resultados
conteos = resultado.get_counts()

# Mostrar los resultados en un histograma
print("Resultados de la medición:")
print(conteos)
plot_histogram(conteos)
plt.show()


ImportError: cannot import name 'Aer' from 'qiskit' (c:\Users\javit\Documents\UNAL\LP02_Tareas\Tarea11\.venv\Lib\site-packages\qiskit\__init__.py)